In [1]:
import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect(':memory:')

In [3]:
table_names = [
    'Categories',   # категории товаров
    'Customers',    # заказчики
    'Employees',    # сотрудники
    'OrderDetails', # детали заказов
    'Orders',       # заказы
    'Products',     # товары
    'Shippers',     # перевозчики
    'Suppliers',    # поставщики
]

In [4]:
for table in table_names:
    data = pd.read_csv(f'{table}.csv', sep = ';')
    data.to_sql(table, con, index=False, if_exists='replace')

In [5]:
pd.read_sql(
    '''
    SELECT * FROM Categories
    ''',
    con,
)

,CategoryID,CategoryName,Description
0,1,Beverages,"Soft drinks, coffees, teas, beers, and ales"
1,2,Condiments,"Sweet and savory sauces, relishes, spreads, an..."
2,3,Confections,"Desserts, candies, and sweet breads"
3,4,Dairy Products,Cheeses
4,5,Grains/Cereals,"Breads, crackers, pasta, and cereal"
5,6,Meat/Poultry,Prepared meats
6,7,Produce,Dried fruit and bean curd
7,8,Seafood,Seaweed and fish


In [6]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выберите заказчиков из Германии, Франции и Мадрида, выведите их название, страну и адрес.

In [7]:
tasks+=1
pd.read_sql(
    '''
    SELECT CustomerName, Country, Address  
    FROM Customers
    WHERE Country in ('Germany','France') or City = 'Madrid'
    ''',
    con,
)

,CustomerName,Country,Address
0,Alfreds Futterkiste,Germany,Obere Str. 57
1,Blauer See Delikatessen,Germany,Forsterstr. 57
2,Blondel père et fils,France,"24, place Kléber"
3,Bólido Comidas preparadas,Spain,"C/ Araquil, 67"
4,Bon app',France,"12, rue des Bouchers"
5,Drachenblut Delikatessend,Germany,Walserweg 21
6,Du monde entier,France,"67, rue des Cinquante Otages"
7,FISSA Fabrica Inter. Salchichas S.A.,Spain,"C/ Moralzarzal, 86"
8,Folies gourmandes,France,"184, chaussée de Tournai"
9,Frankenversand,Germany,Berliner Platz 43


2. Выберите топ 3 страны по количеству заказчиков, выведите их названия и количество записей.

In [8]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, count(CustomerName) as CountCustomers FROM Customers
    GROUP BY Country
    ORDER BY CountCustomers DESC
    LIMIT 3
    ''',
    con,
)

,Country,CountCustomers
0,USA,13
1,Germany,11
2,France,11


3. Выберите перевозчика, который отправил 10-й по времени заказ, выведите его название, и дату отправления.

In [9]:
tasks+=1
pd.read_sql(
    '''
    SELECT ShipperName, OrderDate  
    FROM Orders
    JOIN Shippers
    on Orders.ShipperID = Shippers.ShipperID
    LIMIT 9, 1
    ''',
    con,
)

,ShipperName,OrderDate
0,Federal Shipping,16.07.1996


4. Выберите самый дорогой заказ, выведите список товаров с их ценами.

In [10]:
tasks+=1
pd.read_sql(
    '''
    WITH OrdersDetailsJProducts AS
        (SELECT OrderID, OrderDetails.ProductID, Quantity, ProductName, Price  
        FROM OrderDetails
        JOIN Products
        on OrderDetails.ProductID = Products.ProductID
        ),
    TotalSumOrders AS (
        SELECT OrderID, SUM(Quantity * Price) as Total
        FROM OrdersDetailsJProducts
        GROUP BY OrderID
                        )
        
    SELECT OrderID, ProductName, Quantity, Price
    FROM OrdersDetailsJProducts
    WHERE OrderID=
        (SELECT OrderID 
        FROM TotalSumOrders
        WHERE Total = (
                        SELECT MAX(Total)
                        FROM TotalSumOrders
                       )
        )
    ''',
    con,
)

,OrderID,ProductName,Quantity,Price
0,10372,Sir Rodney's Marmalade,12,81.0
1,10372,Côte de Blaye,40,263.5
2,10372,Camembert Pierrot,70,34.0
3,10372,Mozzarella di Giovanni,42,34.8


5. Какой товар больше всего заказывали по количеству единиц товара, выведите его название и количество единиц в каждом из заказов.

In [11]:
tasks+=1
pd.read_sql(
    '''
    WITH OrdersDetailsJProducts AS(
    SELECT OrderID, OrderDetails.ProductID, Quantity, ProductName
    FROM OrderDetails
    JOIN Products
    on OrderDetails.ProductID = Products.ProductID
    ),
    TotalCountProducts AS (
    SELECT ProductID, SUM(Quantity) as TotalQuantity
    FROM OrdersDetailsJProducts
    GROUP BY ProductID
                          )
                       
    SELECT ProductName, Quantity, OrderID  
    FROM OrdersDetailsJProducts
    WHERE ProductID =(
                      SELECT ProductID
                      FROM TotalCountProducts
                      WHERE TotalQuantity = (
                                            SELECT MAX(TotalQuantity)
                                            FROM TotalCountProducts
                                            )
                      )
    ORDER BY Quantity DESC
    ''',
    con,
)

,ProductName,Quantity,OrderID
0,Gorgonzola Telino,70,10359
1,Gorgonzola Telino,60,10390
2,Gorgonzola Telino,56,10342
3,Gorgonzola Telino,42,10376
4,Gorgonzola Telino,40,10272
5,Gorgonzola Telino,32,10393
6,Gorgonzola Telino,30,10356
7,Gorgonzola Telino,30,10374
8,Gorgonzola Telino,25,10335
9,Gorgonzola Telino,20,10253


6. Выведите топ 5 поставщиков по количеству заказов, выведите их названия, страну, контактное лицо и телефон.

In [12]:
tasks+=1
pd.read_sql(
    '''
    SELECT SupplierName, Country, ContactName, Phone
    FROM Suppliers
    WHERE SupplierID IN (
        SELECT SupplierID
        FROM OrderDetails
        JOIN Products
        on OrderDetails.ProductID = Products.ProductID
        GROUP BY SupplierID
        ORDER BY count(OrderID) DESC
        LIMIT 5
                        )
    ''',
    con,
)

,SupplierName,Country,ContactName,Phone
0,"Pavlova, Ltd.",Australia,Ian Devling,(03) 444-2343
1,"Specialty Biscuits, Ltd.",UK,Peter Wilson,(161) 555-4448
2,Plutzer Lebensmittelgroßmärkte AG,Germany,Martin Bein,(069) 992755
3,Formaggi Fortini s.r.l.,Italy,Elio Rossi,(0544) 60323
4,Norske Meierier,Norway,Beate Vileid,(0)2-953010


7. Какую категорию товаров заказывали больше всего по стоимости в Бразилии, выведите страну, название категории и сумму.

In [13]:
tasks+=1
pd.read_sql(
    '''
    SELECT Country, CategoryName, SUM(Quantity*Price) as Total
    FROM OrderDetails
    JOIN Orders
    ON OrderDetails.OrderID = Orders.OrderID
    JOIN Customers
    ON Orders.CustomerID = Customers.CustomerID
    JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    JOIN Categories
    ON Products.CategoryID = Categories.CategoryID
    WHERE Country = 'Brazil'
    GROUP BY CategoryName
    ORDER BY Total DESC
    LIMIT 1
    ''',
    con,
)

,Country,CategoryName,Total
0,Brazil,Beverages,13690.0


8. Какая разница в стоимости между самым дорогим и самым дешевым заказом из США.

In [14]:
tasks+=1
pd.read_sql(
    '''
    WITH TotalOrdersUSA AS (SELECT SUM(Quantity*Price) as Total
    FROM OrderDetails
    JOIN Products
    ON OrderDetails.ProductID = Products.ProductID
    WHERE OrderID IN(
        SELECT OrderID
        FROM Orders
        WHERE CustomerID in (
            SELECT CustomerID 
            FROM Customers
            WHERE Country = 'USA'
                            )
                    )
    GROUP BY OrderID
    )
    SELECT MAX(Total) - MIN(Total)
    FROM TotalOrdersUSA
    ''',
    con,
)

,MAX(Total) - MIN(Total)
0,7638.45


9. Выведите количество заказов у каждого их трех самых молодых сотрудников, а также имя и фамилию во второй колонке.

In [15]:
tasks+=1
pd.read_sql(
    '''
    WITH ImprovedEmployees AS (
        SELECT EmployeeID,  
                substr(BirthDate, 7) || '-' || substr(BirthDate, 4, 2) || '-' || substr(BirthDate, 1, 2) as DateBirth
        FROM Employees
        ORDER BY DateBirth DESC
        LIMIT 3)
    
    SELECT COUNT() as QuantityOrders, FirstName || ' ' || LastName as FirstLastName
    FROM Orders
    JOIN Employees
    ON Orders.EmployeeID = Employees.EmployeeID
    WHERE Employees.EmployeeID IN 
        (SELECT EmployeeID
         FROM ImprovedEmployees
        )
    GROUP BY Employees.EmployeeID
    ORDER BY QuantityOrders DESC
    ''',
    con,
)

,QuantityOrders,FirstLastName
0,31,Janet Leverling
1,29,Nancy Davolio
2,6,Anne Dodsworth


10. Сколько банок крабового мяса всего было заказано.

In [16]:
tasks+=1
pd.read_sql(
    '''
    SELECT ProductName, SUM(Quantity) 
    FROM OrderDetails
    JOIN Products
    on OrderDetails.ProductID = Products.ProductID
    WHERE ProductName = 'Boston Crab Meat'
    ''',
    con,
)

,ProductName,SUM(Quantity)
0,Boston Crab Meat,256


In [17]:
if tasks==10:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')

Выполнены все задания 🙂
